<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_plant_functional_groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [2]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’




# Source

## Database Connection

In [3]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [4]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [5]:
billing <- bq_test_project()

## Database Query

### gridVeg_plant_intercepts

In [6]:
sql_plant_intercepts <- 
"
  SELECT 
    survey_ID,
    grid_point,
    key_plant_code,
    plant_native_status,
    plant_life_cycle,
    plant_life_form,
    intercepts_pct
  FROM 
    `mpg-data-warehouse.vegetation_gridVeg_summaries.gridVeg_plant_intercepts`
"

In [7]:
bq_plant_intercepts <- bq_project_query(billing, sql_plant_intercepts)

In [8]:
tb_plant_intercepts <- bq_table_download(bq_plant_intercepts)

In [9]:
df_plant_intercepts <- as.data.frame(tb_plant_intercepts)

In [39]:
head(df_plant_intercepts, n=4)

,survey_ID,grid_point,key_plant_code,plant_native_status,plant_life_cycle,plant_life_form,intercepts_pct
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,436,1,HEUCYL,native,perennial,forb,2.5
2,436,1,ALLCER,native,perennial,forb,0.5
3,436,1,GEUTRI,native,perennial,forb,1.0
4,436,1,ERIG_SP,native,unknown,forb,0.5


### gridVeg_survey_metadata

In [35]:
sql_survey_metadata <- "
  SELECT 
    survey_ID,
    year,
    survey_sequence
  FROM
    mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata
"

In [36]:
bq_survey_metadata <- bq_project_query(billing, sql_survey_metadata)

In [37]:
tb_survey_metadata <- bq_table_download(bq_survey_metadata)

In [38]:
df_survey_metadata <- as.data.frame(tb_survey_metadata)

### location_position_classification

In [44]:
sql_position_class <- "
  SELECT
    grid_point,
    aspect_mean_deg,
    elevation_mean_m,
    slope_mean_deg,
    cover_type_2016_gridVeg,
    type3_vegetation_indicators,
    type4_indicators_history
  FROM
    `mpg-data-warehouse.grid_point_summaries.location_position_classification` 
"

In [45]:
bq_position_class <- bq_project_query(billing, sql_position_class)

In [46]:
tb_position_class <- bq_table_download(bq_position_class)

In [47]:
df_position_class <- as.data.frame(tb_position_class)

# Wrangle

Start with the view vegetation_gridVeg_summaries:gridVeg_plant_intercepts. Remove records where key_plant_code = “NV” (corresponds with key_plant_species = 360).

## Explore key_plant_code "NV"

In [ ]:
df_plant_intercepts %>%
  filter(key_plant_code == "NV")

## Remove key_plant_code "NV"

In [18]:
df_plant_functional_groups <- df_plant_intercepts %>%
  filter(key_plant_code != "NV")

## Recode

Recode the levels of plant_life_cycle to simplify them (re-coded values are supplied in the Readme).


In [19]:
df_plant_functional_groups <- df_plant_functional_groups %>%
  mutate(plant_life_cycle = ifelse(plant_life_cycle == "biennial perennial" |
                                   plant_life_cycle == "annual perennial" |
                                   plant_life_cycle == "annual biennial perennial" |
                                   plant_life_cycle == "annual biennial"
                                   , "multiple", plant_life_cycle))

## Calculate Detection Rate

Group the data on {survey_ID, plant_native_status, plant_life_cycle, plant_life_form}. Sum the intercepts_pct within each group, and divide the sums by 4 to obtain the detection_rate per 100 intercepts possible. 

In [20]:
df_plant_functional_groups %>%
  group_by(survey_ID, plant_native_status, plant_life_cycle, plant_life_form) %>%
  summarise(intercepts_pct_sum = sum(intercepts_pct), detection_rate = sum(intercepts_pct)/4) %>%
  filter(survey_ID == "436")

`summarise()` regrouping output by 'survey_ID', 'plant_native_status', 'plant_life_cycle' (override with `.groups` argument)



survey_ID,plant_native_status,plant_life_cycle,plant_life_form,intercepts_pct_sum,detection_rate
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
436,native,annual,forb,3.5,0.875
436,native,multiple,forb,1.0,0.250
436,native,perennial,forb,8.5,2.125
436,native,perennial,graminoid,44.5,11.125
436,native,perennial,shrub,10.5,2.625
436,native,perennial,tree,2.0,0.500
436,native,unknown,forb,0.5,0.125
436,nonnative,annual,forb,1.5,0.375
436,nonnative,annual,graminoid,5.5,1.375


In [21]:
df_plant_functional_groups <- df_plant_functional_groups %>%
  group_by(survey_ID, plant_native_status, plant_life_cycle, plant_life_form) %>%
  summarise(intercepts_pct_sum = sum(intercepts_pct), detection_rate = sum(intercepts_pct)/4)

`summarise()` regrouping output by 'survey_ID', 'plant_native_status', 'plant_life_cycle' (override with `.groups` argument)



In [22]:
df_plant_functional_groups %>% glimpse()

Rows: 9,015
Columns: 6
Groups: survey_ID, plant_native_status, plant_life_cycle [5,560]
$ survey_ID           <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5FAD…
$ plant_native_status <chr> "native", "native", "native", "native", "native",…
$ plant_life_cycle    <chr> "annual", "multiple", "perennial", "perennial", "…
$ plant_life_form     <chr> "forb", "forb", "forb", "graminoid", "shrub", "fo…
$ intercepts_pct_sum  <dbl> 6.5, 1.0, 14.5, 5.0, 8.5, 5.0, 46.0, 15.0, 25.0, …
$ detection_rate      <dbl> 1.625, 0.250, 3.625, 1.250, 2.125, 1.250, 11.500,…


Then, make sure all combinations of functional groups that are found in the data are represented in each survey_ID. For those groups which were not detected at a survey_ID, fill the detection_rate with 0. The following tables show the categories of functional groups and the full list of all possible combinations.

## Group Fill

In [23]:
df_plant_functional_groups <- df_plant_functional_groups %>%
  ungroup() %>%
  complete(survey_ID, 
           nesting(plant_native_status, plant_life_cycle, plant_life_form), 
           fill = list(intercepts_pct_sum = 0, detection_rate = 0))

In [26]:
df_plant_functional_groups %>% head()

survey_ID,plant_native_status,plant_life_cycle,plant_life_form,intercepts_pct_sum,detection_rate
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,native,annual,forb,6.5,1.625
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,native,annual,graminoid,0.0,0.000
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,native,biennial,forb,0.0,0.000
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,native,multiple,forb,1.0,0.250
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,native,multiple,graminoid,0.0,0.000
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,native,perennial,forb,14.5,3.625


## Join Metadata

Finally, join the metadata from requested tables or views and complete the schema below. There are 1244 survey IDs, with 2010 data removed. With 30 existing combinations of plant functional groups in the data, this means that the view should contain 1244 * 30 = 37,320 rows when done (in 2020). 

In [61]:
# bring gridpoints back in
grid_point_ref <- df_plant_intercepts %>%
  select(survey_ID, grid_point) %>%
  distinct(survey_ID, grid_point)

df_plant_functional_groups <- df_plant_functional_groups %>%
  left_join(grid_point_ref)

In [69]:
df_plant_functional_groups <- df_plant_functional_groups %>%
  left_join(df_survey_metadata) %>%
  glimpse()

Joining, by = "survey_ID"



Rows: 31,050
Columns: 9
$ survey_ID           <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5FAD…
$ plant_native_status <chr> "native", "native", "native", "native", "native",…
$ plant_life_cycle    <chr> "annual", "annual", "biennial", "multiple", "mult…
$ plant_life_form     <chr> "forb", "graminoid", "forb", "forb", "graminoid",…
$ intercepts_pct_sum  <dbl> 6.5, 0.0, 0.0, 1.0, 0.0, 14.5, 5.0, 8.5, 0.0, 0.0…
$ detection_rate      <dbl> 1.625, 0.000, 0.000, 0.250, 0.000, 3.625, 1.250, …
$ grid_point          <int> 285, 285, 285, 285, 285, 285, 285, 285, 285, 285,…
$ year                <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2…
$ survey_sequence     <chr> "2016", "2016", "2016", "2016", "2016", "2016", "…


In [72]:
df_plant_functional_groups <- df_plant_functional_groups %>%
  left_join(df_position_class) %>%
  glimpse()

Joining, by = "grid_point"



Rows: 31,050
Columns: 15
$ survey_ID                   <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "…
$ plant_native_status         <chr> "native", "native", "native", "native", "…
$ plant_life_cycle            <chr> "annual", "annual", "biennial", "multiple…
$ plant_life_form             <chr> "forb", "graminoid", "forb", "forb", "gra…
$ intercepts_pct_sum          <dbl> 6.5, 0.0, 0.0, 1.0, 0.0, 14.5, 5.0, 8.5, …
$ detection_rate              <dbl> 1.625, 0.000, 0.000, 0.250, 0.000, 3.625,…
$ grid_point                  <int> 285, 285, 285, 285, 285, 285, 285, 285, 2…
$ year                        <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016,…
$ survey_sequence             <chr> "2016", "2016", "2016", "2016", "2016", "…
$ aspect_mean_deg             <dbl> 138.749, 138.749, 138.749, 138.749, 138.7…
$ elevation_mean_m            <dbl> 1352.10, 1352.10, 1352.10, 1352.10, 1352.…
$ slope_mean_deg              <dbl> 24.5141, 24.5141, 24.5141, 24.5141, 24.51…
$ cover_type_2016_gridVeg  

# Output

In [73]:
write_csv(df_plant_functional_groups, path = "gridVeg_plant_functional_groups-WRANGLE.csv")